In [1]:
# import libraries
!pip install ipython-sql
import pandas as pd
import sqlite3
print('current_SQLite_Version : ',sqlite3.sqlite_version)

You should consider upgrading via the 'C:\Users\Assem Salama\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


current_SQLite_Version :  3.37.2


In [2]:
# import libraries
import pandas as pd
import sqlite3
print('Updated_SQLite Version : ',sqlite3.sqlite_version)

Updated_SQLite Version :  3.37.2


In [3]:
# 1st : Read information dataset & explore it:
import pandas as pd
info_df = pd.read_excel('Dataset1.xlsx')
print(info_df.info())
print(info_df.columns)
info_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   District   641 non-null    object
 1   State      641 non-null    object
 2   Growth     641 non-null    object
 3   Sex_Ratio  641 non-null    object
 4   Literacy   641 non-null    object
dtypes: object(5)
memory usage: 25.2+ KB
None
Index(['District', 'State ', 'Growth', 'Sex_Ratio', 'Literacy'], dtype='object')


,District,State,Growth,Sex_Ratio,Literacy
0,South Andaman,Andaman And Nicobar Islands,0.1423,871.0,89.13
1,North And Middle Andaman,Andaman And Nicobar Islands,-0.0002,925.0,83.91
2,Nicobars,Andaman And Nicobar Islands,-0.1242,777.0,78.06


>**Issues:**
1. Growth ,Sex_Ratio ,Literacy cols need to be converted as float dtype.
2. 'State ' col should convert 'State' `Remove the white-space`.
[This was the root reason of this error : database error: no such 'State'column.]

In [4]:
# 2nd : Read population dataset & explore it:
pop_df = pd.read_excel('Dataset2-Population.xlsx')
print(pop_df.info())
print(pop_df.columns)
pop_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   District    640 non-null    object 
 1   State       607 non-null    object 
 2   Area_km2    640 non-null    float64
 3   Population  640 non-null    float64
dtypes: float64(2), object(2)
memory usage: 20.1+ KB
None
Index(['District', 'State', 'Area_km2', 'Population'], dtype='object')


,District,State,Area_km2,Population
0,Adilabad,Andhra Pradesh,16105.0,2741239.0
1,Agra,Uttar Pradesh,4041.0,4418797.0
2,Ahmadabad,Gujarat,8107.0,7214225.0
3,Ahmadnagar,Maharashtra,17048.0,4543159.0
4,Aizawl,Mizoram,3576.0,400309.0


In [5]:
# 3rd : Data Wrangling step:
# Convert the columns types in 'info_df' from object to float.
info_df['Growth'] = pd.to_numeric(info_df['Growth'], errors='coerce')
info_df['Sex_Ratio'] = pd.to_numeric(info_df['Sex_Ratio'], errors='coerce')
info_df['Literacy'] = pd.to_numeric(info_df['Literacy'], errors='coerce')

# Edit 'State ' column to be 'State' ,Removing WhiteSpace.
info_df.rename(columns = {'State ':'State'} , inplace=True)

In [6]:
# 4th : create SQL database:
# Create database called 'indian_census'
db_connect = sqlite3.connect('indian_census')
# Cursors allow us to excute SQL_queries with DB.
cur = db_connect.cursor()
# create tables "info_table" & "pop_table" in the database.
cur.execute("create table if not exists info_table ('District' object not null primary key,'State' object,'Growth' float64,'Sex_Ratio' float64,'Literacy' float64)")
cur.execute("create table if not exists pop_table  ('District' object not null primary key,'State' object,'Area_km2' float64,'Population' float64)")

# show all the new tables in the newly created SQL Database.
cur.execute("SELECT name from sqlite_master where type='table' order by name;")
results = cur.fetchall()
results

[('info_table',), ('pop_table',)]

In [7]:
# 5th : importing our data into 'indian_census'SQL Database:
# use pandas to run SQL querry, allows us to save our data into SQL database.
import pandas as pd
info_df.to_sql('info_table' , db_connect , if_exists='replace' , index = False)
pop_df.to_sql('pop_table' , db_connect , if_exists='replace' ,  index = False)

## ✳️ Answering some Basic Quastions:

In [8]:
# 1) Show the first 3 rows in the info_table:
from numpy import select
first_3_rows = pd.read_sql_query('select *from info_table limit 3',
                      db_connect)
first_3_rows

,District,State,Growth,Sex_Ratio,Literacy
0,South Andaman,Andaman And Nicobar Islands,0.1423,871.0,89.13
1,North And Middle Andaman,Andaman And Nicobar Islands,-0.0002,925.0,83.91
2,Nicobars,Andaman And Nicobar Islands,-0.1242,777.0,78.06


In [9]:
# 2) The num of rows in each table:
info_no = pd.read_sql_query('''select count(*) num_of_rows
                               from info_table
                            ''',db_connect)

pop_no = pd.read_sql_query('''select count(*) num_of_rows
                              from pop_table
                           ''',db_connect)
print(info_no)
print(pop_no)

   num_of_rows
0          641
   num_of_rows
0          640


In [10]:
# 3) Show me the data points related to State of Jharkhand & Bihar:
pd.read_sql_query('''select * 
                     from info_table
                     where State in ('Jharkhand','Bihar')
                  ''',db_connect).head()

,District,State,Growth,Sex_Ratio,Literacy
0,Patna,Bihar,0.2373,897.0,70.68
1,Purbi Champaran,Bihar,0.2943,902.0,55.79
2,Muzaffarpur,Bihar,0.2814,900.0,63.43
3,Madhubani,Bihar,0.2551,926.0,58.62
4,Gaya,Bihar,0.2643,937.0,63.67


In [11]:
# 4) Calculate me the population of india:
Total_population = pd.read_sql_query('''select sum(Population) Total_population
                     from pop_table
                  ''',db_connect)

print(Total_population.iloc[0,0])
Total_population

1210854977.0


,Total_population
0,1.210855e+09


In [12]:
# 5) Tell me the average growth Rate of india:
pd.read_sql_query('''select avg(Growth*100) Avg_growth
                     from info_table
                  ''',db_connect)

,Avg_growth
0,19.245922


In [13]:
# 6) What is the average growth rate for each state?
pd.read_sql_query('''select State,avg(Growth*100) Avg_growth
                     from info_table
                     group by State
                     order by Avg_growth Desc
                     limit 3
                  ''',db_connect)

,State,Avg_growth
0,Nagaland,82.280909
1,Dadra and Nagar Haveli,55.880000
2,Daman and Diu,42.740000


In [14]:
# 7) What is the avg Sex_Ratio for each state?
pd.read_sql_query('''select State, round(avg(Sex_Ratio),2) avg_sexRatio
                     from info_table
                     group by State
                     order by avg_sexRatio desc
                     limit 3
                  ''',db_connect)

,State,avg_sexRatio
0,Kerala,1080.43
1,Puducherry,1074.50
2,Uttarakhand,1010.46


In [15]:
# 8) What is the avg_Literacy for each state ? and show me the states with score more than 90.
pd.read_sql_query('''select State, round(avg(Literacy),0) avg_literacy
                     from info_table 
                     group by State
                     having avg_literacy >90
                     order by avg_literacy desc
                  ''',db_connect)

,State,avg_literacy
0,Kerala,94.0
1,Lakshadweep,92.0


In [16]:
# 9) show me the top 3 state with highest growth Rate:
pd.read_sql_query('''select state , round(avg(Growth*100),0) avg_growth
                     from info_table
                     group by State
                     order by avg_growth desc
                     limit 3
                  ''',db_connect)

,State,avg_growth
0,Nagaland,82.0
1,Dadra and Nagar Haveli,56.0
2,Daman and Diu,43.0


In [17]:
# 10) show me the buttom 3 state with lowest sex Rate:
pd.read_sql_query('''select state , avg(Sex_Ratio) avg_sex_Ratio
                     from info_table
                     group by State
                     order by avg_sex_Ratio asc
                     limit 4
                  ''',db_connect)

,State,avg_sex_Ratio
0,State,NaN
1,Dadra and Nagar Haveli,774.0
2,Daman and Diu,782.5
3,Chandigarh,818.0


In [18]:
# 11) Show me top & buttom state at literacy_Ratio both in one single table.
# 1st: Create topstates Table that contain top 3 states with highest 'Literacy_Ratio':
cur.execute("create table if not exists topstates  ('State' object,'topstates' float64)")
top_3_states_df = pd.read_sql_query(''' select state , round(avg(Literacy),0) avg_literacy
                                        from info_table
                                        group by State
                                        order by avg_literacy desc
                                        limit 3;''',db_connect)
top_3_states_df.to_sql('topstates',db_connect,if_exists='replace' ,  index =False)

# 2nd: Create topstates Table that contain buttom 3 states with lowest 'Literacy_Ratio':
cur.execute("create table if not exists buttomstates  ('State' object,'buttomstates' float64)")
buttom_3_states_df = pd.read_sql_query(''' select state , round(avg(Literacy),0) avg_literacy
                                           from info_table
                                           group by State
                                           order by avg_literacy asc
                                           limit 4;''',db_connect).drop(index=0)
buttom_3_states_df.to_sql('buttomstates',db_connect,if_exists='replace' , index =False)

# 3rd: Show each Table separately ,then display them in single table using 'Union-operator':
print(pd.read_sql_query('select * from buttomstates',db_connect),"\n\n",
      pd.read_sql_query('select * from topstates',db_connect))
pd.read_sql_query(''' select * from (topstates)a
                      union
                      select * from (buttomstates)b
                      order by avg_literacy desc;
                  ''',db_connect)

               State  avg_literacy
0              Bihar          62.0
1  Arunachal Pradesh          64.0
2  Jammu and Kashmir          65.0 

          State  avg_literacy
0       Kerala          94.0
1  Lakshadweep          92.0
2      Mizoram          89.0


,State,avg_literacy
0,Kerala,94.0
1,Lakshadweep,92.0
2,Mizoram,89.0
3,Jammu and Kashmir,65.0
4,Arunachal Pradesh,64.0
5,Bihar,62.0


In [19]:
# 12) Tell me all the states starting with letter A or B:
pd.read_sql_query(''' select distinct state
                      from info_table
                      where (lower(state) like 'a%') or (lower(state) like 'b%') or
                            (lower(state) like 'm%' and lower(state) like '%a');
                  ''',db_connect)

,State
0,Andaman And Nicobar Islands
1,Andhra Pradesh
2,Arunachal Pradesh
3,Assam
4,Bihar
5,Maharashtra
6,Meghalaya


- **Population =** NO.Males + NO.Females ...1
- **Sex_Ratio =** NO.Females / NO.Males ...2
 
- From eq(1) we can tell `Females = pop - Males`.
- From eq(2) we can tell `Females = Sex_Ratio * Males`.
-  SO, `pop - Males = Sex_Ratio * Males`.

 ### **pop = Males(Sex_Ratio + 1)**
 >### **Males = pop/(Sex_Ratio + 1)**
 >### **Females = pop - (pop/(Sex_Ratio + 1))**

In [20]:
# 13) What are the num of male\female with respect to total population for each State?
# To answer this question we need to join the info_table where (sex_ratio AKA Gender) exists and,
# pop_table where (populatio) exists:
pd.read_sql_query('''select d.State , sum(d.Males) total_males, sum(d.Females) total_females
                     from
                        (select c.District ,c.State ,c.Population,
                                round((c.Population/(c.Sex_Ratio+1))) Males ,
                                round((c.Population - (c.Population/(c.Sex_Ratio+1)))) Females
                         from
                            (select a.District, a.State, (a.Sex_Ratio/1000) Sex_Ratio,b.Population
                              from (info_table) a 
                              inner join (pop_table) b 
                              on a.District = b.District) c)d
                     group by d.State
                  ''',db_connect).head(3)

,State,total_males,total_females
0,Andaman And Nicobar Islands,148014.0,126970.0
1,Andhra Pradesh,42439617.0,42141160.0
2,Arunachal Pradesh,624752.0,582402.0


### **Literacy_Ratio = NO.literate/pop**
> ### **NO.literate = Literacy_Ratio * pop**
> ### **NO.illiterate = pop * (1-Literacy_Ratio)**

In [21]:
# 14) What are the num of literate\illiterate with respect to total population for each State?
pd.read_sql_query(''' select d.State ,d.Population, sum(d.literate) NO_literate ,
                                                    sum(d.illiterate) NO_illiterate
                      from
                          (select c.District , c.State , c.Population,
                                round((c.Literacy_Ratio * c.Population)) literate,
                                round(c.Population*(1-c.Literacy_Ratio)) illiterate
                           from
                              (select a.District,a.State,(a.Literacy/100) Literacy_Ratio,b.Population
                              from (info_table) a
                              inner join (pop_table) b
                              on a.District = b.District) c) d
                      group by d.State
                  ''',db_connect).head(3)

,State,Population,NO_literate,NO_illiterate
0,Andaman And Nicobar Islands,238142.0,241015.0,33969.0
1,Andhra Pradesh,5296741.0,56671677.0,27909100.0
2,Arunachal Pradesh,148226.0,761557.0,445597.0


### **Population  = previous_Census + (Growth*previous_Census)**
> ### **previous_Census = Population / (1 + Growth)**

In [22]:
# 15) What is the population in the previous census and average growth percent for india and per state?
pd.read_sql_query('''select d.State , sum(d.previous_census) previous_census,
                            avg(d.Growth_percent) avg_Growth_percent,sum(d.current_pop) current_pop
                          from
                            (select c.District , c.State , round((c.Population / (1+ c.Growth))) previous_census,
                                    c.Growth*100 Growth_percent,c.Population current_pop
                              from
                                  (select a.District , a.State ,a.Growth,b.population
                                  from (info_table) a
                                  inner join (pop_table) b
                                  on a.District = b.District) c) d
                          group by d.State
                          order by avg_Growth_percent desc
                   ''',db_connect).head(4)

,State,previous_census,avg_Growth_percent,current_pop
0,Nagaland,1047139.0,82.280909,1978502.0
1,Daman and Diu,158207.0,42.740000,243247.0
2,Puducherry,974363.0,34.297500,1247953.0
3,Meghalaya,2126073.0,29.750000,2708049.0


In [23]:
pd.read_sql_query('''select round(sum(e.previous_census)) india_previous_pop , round(sum(e.current_pop)) india_current_pop,
                            avg(avg_Growth_percent) india_avg_Growth_percent
                     from       
                        (select d.State , avg(d.Growth_percent) avg_Growth_percent,
                                sum(d.previous_census) previous_census ,sum(d.current_pop) current_pop
                          from
                            (select c.District , c.State , round((c.Population / (1+ c.Growth))) previous_census,
                                    c.Growth*100 Growth_percent,c.Population current_pop
                              from
                                  (select a.District , a.State ,a.Growth,b.population
                                  from (info_table) a
                                  inner join (pop_table) b
                                  on a.District = b.District) c) d
                          group by d.State
                          order by avg_Growth_percent desc) e
                   ''',db_connect)

,india_previous_pop,india_current_pop,india_avg_Growth_percent
0,1.005585e+09,1.183964e+09,18.984364


In [24]:
# 16) What is the impact of the gradual increament of population VS. country's area?
pd.read_sql_query('''select j.india_previous_pop , j.india_avg_Growth_percent , 
                            j.india_current_pop ,
                           (j.india_total_Area/j.india_previous_pop)previous_area_per_capita,
                           (j.india_total_Area/j.india_current_pop)current_area_per_capita
                     from
                        (select h.* , i.*
                        from
                            (select '1' as key ,f.*
                            from
                                (select round(sum(e.previous_census)) india_previous_pop , round(sum(e.current_pop)) india_current_pop,
                                        avg(avg_Growth_percent) india_avg_Growth_percent
                                from       
                                    (select d.State , avg(d.Growth_percent) avg_Growth_percent,
                                            sum(d.previous_census) previous_census ,sum(d.current_pop) current_pop
                                      from
                                        (select c.District , c.State , round((c.Population / (1+ c.Growth))) previous_census,
                                                c.Growth*100 Growth_percent,c.Population current_pop
                                          from
                                              (select a.District , a.State ,a.Growth,b.population
                                              from (info_table) a
                                              inner join (pop_table) b
                                              on a.District = b.District) c) d
                                      group by d.State
                                      order by avg_Growth_percent desc) e) f) h
                          inner join
                              (select '1' as key ,g.*
                              from
                                  (select sum(Area_km2) india_total_Area
                                  from pop_table) g) i
                          on h.key = i.key) j
                   ''',db_connect)

,india_previous_pop,india_avg_Growth_percent,india_current_pop,previous_area_per_capita,current_area_per_capita
0,1.005585e+09,18.984364,1.183964e+09,0.003149,0.002675


In [32]:
# create run() function to run query easily.
def run(query):
    return pd.read_sql_query(query,db_connect)

In [43]:
# 17) show me the top 3 District in each state with highest Literacy_Rate.

query = '''
select a.*
from 
    (select
     *, rank() over (partition by State order by Literacy desc) as Rank
     from
     info_table) a

where a.Rank in (1,2,3)
'''
run(query).head(10)

,District,State,Growth,Sex_Ratio,Literacy,Rank
0,South Andaman,Andaman And Nicobar Islands,0.1423,871.0,89.13,1
1,North And Middle Andaman,Andaman And Nicobar Islands,-0.0002,925.0,83.91,2
2,Nicobars,Andaman And Nicobar Islands,-0.1242,777.0,78.06,3
3,Hyderabad,Andhra Pradesh,0.0297,954.0,83.25,1
4,Rangareddy,Andhra Pradesh,0.4816,961.0,75.87,2
5,West Godavari,Andhra Pradesh,0.0351,1004.0,74.63,3
6,Papumpare,Arunachal Pradesh,0.4473,980.0,79.95,1
7,Lower Subansiri,Arunachal Pradesh,0.4900,984.0,74.35,2
8,East Siang,Arunachal Pradesh,0.1352,980.0,72.54,3
9,Kamrup Metropolitan,Assam,0.1834,936.0,88.71,1
